# <center>Answering Business Questions With SQL</center>
<div>
<img src="Data/chinook-schema.svg" width="400"/>
</div>
In this project we are going to be answering questions related to the sale of music from a fictional music shop, akin to iTunes

The [Chinook database](https://github.com/lerocha/chinook-database) contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables. The above schema diagram helps us understand the available columns and the structure of the data.

We are going to use SQL explore and analyse:
- The most popular genres of music in the United States of America.
- Sales performance of Chinook sales representatives.
- Sales by country.
- The purchasing of albums versus individual song tracks.

## Importing SQL

In [1]:
# Importing SQLite3 
import sqlite3 as sql

## Importing Chinook Database for use in this project

In [2]:
# Loading the Chinook database, which we have stored in the Data subdirectory.
%load_ext sql
# Loading the factbook database
%sql sqlite:///Data/chinook.db

'Connected: @Data/chinook.db'

# Exploring Dataset

Exploring the Chinook database, first by displaying the master table, which shows all the sub-tables in the database:

In [3]:
%%sql
SELECT name,
       type
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///Data/chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


We can see that we have tables which correlate with the schema image at the top of this notebook. We can use the linking columns to combine tables in order to recover all the business information required for this project.

Exploring one of these tables below:

In [4]:
%%sql
SELECT *
  FROM customer
 LIMIT 5;

 * sqlite:///Data/chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


We can modify <mark>SELECT</mark> option in this query to limit the amount of information it returns:

In [5]:
%%sql
SELECT customer_id,
       first_name,
       last_name,
       email
  FROM customer
 LIMIT 5;

 * sqlite:///Data/chinook.db
Done.


customer_id,first_name,last_name,email
1,Luís,Gonçalves,luisg@embraer.com.br
2,Leonie,Köhler,leonekohler@surfeu.de
3,François,Tremblay,ftremblay@gmail.com
4,Bjørn,Hansen,bjorn.hansen@yahoo.no
5,František,Wichterlová,frantisekw@jetbrains.com


## Determining the most popular genres in the United States of America

The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| Artist Name | Genre |
| --- | --- |
| Regal | Hip-Hop |
|Red Tone | Punk |
|Meteor and the Girls |	Pop |
|Slim Jim Bites	| Blues |


The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

We'll need to write a query to find out which genres sell the most tracks in the USA, write up a summary of your findings, and make a recommendation for the three artists whose albums we should purchase for the store.

In [6]:
%%sql

WITH usa_customers AS
    (
    SELECT *
      FROM customer
     WHERE country = 'USA'
    ),
    
    combined_us_invoice_track_data AS
    (
    SELECT *
      FROM usa_customers uc
     INNER JOIN invoice i ON i.customer_id = uc.customer_id
     INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
     INNER JOIN track t ON il.track_id = t.track_id
    ),
    
    track_info_for_analysis AS
    (
    SELECT customer_id,
           name track_name,
           country,
           genre_id
      FROM combined_us_invoice_track_data
    ),
    
    genre_info_for_analysis AS
    (
    SELECT customer_id,
           track_name,
           country,
           g.genre_id genre_id,
           g.name name
      FROM track_info_for_analysis ti
     INNER JOIN genre g ON ti.genre_id = g.genre_id)
    

SELECT name, 
       COUNT(*) total_tracks_sold, 
       ROUND((COUNT(*) / (
                          SELECT CAST(COUNT(*) AS float)
                            FROM genre_info_for_analysis
                          ) * 100), 2) percent_tracks_sold
  FROM genre_info_for_analysis
 GROUP BY genre_id
 ORDER BY 2 DESC;    

 * sqlite:///Data/chinook.db
Done.


name,total_tracks_sold,percent_tracks_sold
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Pop,22,2.09
Latin,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


From the reqults of our query, our genres of interest have the following shares of the sales in the United States:

| Genre | Absolute Sales | Percent of Total Sales |
| --- | --- | --- |
| Hip-Hop | 20 | 1.9% |
| Punk | 130 | 12.4% |
| Pop | 22 | 2.1% |
| Blues | 36 | 3.4% |

Therefore, it would make sense to purchase and promote **Red Tone** (_Punk_), **Slim Jim Bites** (_Blues_), and **Meteor and the Girls** (_Pop_), in our store.

## Analysing sales by support agent

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

We might like to consider whether any extra columns from the employee table explain any variance we see, or whether the variance might instead be indicative of employee performance.

In [7]:
%%sql

WITH invoice_customer AS
    (
    SELECT *
      FROM customer c
      LEFT JOIN invoice i ON c.customer_id = i.customer_id
    ),
    
    invoice_customer_employee AS
    (
    SELECT e.first_name employee_first_name, 
           e.last_name employee_last_name, 
           e.employee_id employee_id,
           total,
           hire_date,
           title,
           invoice_date
      FROM invoice_customer ic
      LEFT JOIN employee e ON e.employee_id = ic.support_rep_id
    )
    
SELECT employee_first_name || " "  || employee_last_name "Sales Support Staff",
       title "Title",
       COUNT(total) "Total Number of Sales",
       ROUND(SUM(total),2) "Total Value of Sales (USD)",
       ROUND(SUM(total) / (
                     SELECT COUNT(total)
                       FROM invoice_customer_employee
                      GROUP BY employee_id
                           ),2) "Average Sale Value (USD)",
       hire_date "Hire Date"
  FROM invoice_customer_employee
 GROUP BY employee_id


 * sqlite:///Data/chinook.db
Done.


Sales Support Staff,Title,Total Number of Sales,Total Value of Sales (USD),Average Sale Value (USD),Hire Date
Jane Peacock,Sales Support Agent,212,1731.51,8.17,2017-04-01 00:00:00
Margaret Park,Sales Support Agent,214,1584.0,7.47,2017-05-03 00:00:00
Steve Johnson,Sales Support Agent,188,1393.92,6.58,2017-10-17 00:00:00


## Observations

We can see that there are three sales support agents. Jane Peacock has the highest value of sales at \\$1,732, while Margaret Park has more sales with 214. This indicates that Jane's orders are higher in value on average (\\$8.17 vs \\$7.47). 

The most important observation is that both the total value of sales and average sales value correlates with the date the sale support agent was hired. Jane Peacock was hired in April and has the highest total sales and highest average sale value, indicative of the additional experience she has. Steve Johnson was hired last, in October, and has the lowest total sales and average sale value, indicative of his lesser experience.

## Analysing sales by country

The next task is to analyze the sales data for customers from each different country. We have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, we have been directed to calculate data, for each country, on the:

- Total number of customers.
- Total value of sales.
- Average value of sales per customer.
- Average order value.

In [8]:
%%sql

WITH country_customers AS
    (
    SELECT country, 
           COUNT(DISTINCT(c.customer_id)) number_customers,
           ROUND(SUM(i.total),2) country_total_sales,
           AVG(i.total) average_order,
           total
      FROM customer c
     INNER JOIN invoice i ON c.customer_id = i.customer_id
     GROUP BY c.country
     ORDER BY number_customers DESC
    ),
    
    country_other AS
    (
    SELECT CASE
               WHEN number_customers = 1 THEN "Other"
               ELSE country
            END country,
           SUM(number_customers) total_customers,
           SUM(country_total_sales) total_sales,
           ROUND(SUM(country_total_sales)/SUM(number_customers),2) sales_per_customer,
           ROUND(average_order, 2) average_order,
            CASE
               WHEN number_customers = 1 THEN 0
               ELSE 1
             END country_or_other
      FROM country_customers
     GROUP BY 1
     ORDER BY 2 DESC)
    
SELECT country Country,
       total_customers "Total Customers",
       total_sales "Total Sales (USD)",
       sales_per_customer "Average Sales Per Customer (USD)",
       average_order "Average Order Value (USD)"
  FROM country_other
ORDER BY country_or_other DESC

 * sqlite:///Data/chinook.db
Done.


Country,Total Customers,Total Sales (USD),Average Sales Per Customer (USD),Average Order Value (USD)
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
France,5,389.07,77.81,7.78
Brazil,5,427.68,85.54,7.01
Germany,4,334.62,83.66,8.16
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Czech Republic,2,273.24,136.62,9.11
Other,15,1094.94,73.0,7.52


## Observations

The USA has the most customers, but the Czech Republic, Portugal and India have the highest average sales per customer. However, we probably should not read into this too much as these countries only have two customers from each. 

## Understanding purchasing of albums vs individual tracks

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [9]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

 * sqlite:///Data/chinook.db
Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


## Observations

We can see that 81% of sales are single tracks, while only 19% are the purchase of full albums. Given this information, the business may decide to place more emphasis on the promotion of popular singles. Alternatively, if the company is looking to sell more albums, it should promote and provide value deals on the albums which it is selling.